<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/PIFuHD_ttmai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Παράγουμε 3D μοντέλα από απλές 2D φωτογραφίες!

## Ολα τα credits στον δημιουργό αυτού του notebook: 
### Δείτε την δουλειά του και περισσότερες λεπτομέρειες για το project στον παρακάτω σύνδεσμο:
#### PIFuHD Demo: https://shunsukesaito.github.io/PIFuHD/

![](https://shunsukesaito.github.io/PIFuHD/resources/images/pifuhd.gif)

Made by [![Follow](https://img.shields.io/twitter/follow/psyth91?style=social)](https://twitter.com/psyth91)

To see how the model works, visit the project repository.

[![GitHub stars](https://img.shields.io/github/stars/facebookresearch/pifuhd?style=social)](https://github.com/facebookresearch/pifuhd)

## Σημείωση
Επιβεβαίωσε ότι έχεις επιλέξει το "GPU" στο μενού: Runtime -> Change Runtime type -> Hardware Accelerator

## Περισσότερες πληροφορίες για αυτό το project
- Paper: https://arxiv.org/pdf/2004.00452.pdf
- Repo: https://github.com/facebookresearch/pifuhd
- Project Page: https://shunsukesaito.github.io/PIFuHD/


## Αντιγράφουμε το "PIFuHD" repository

In [ ]:
!git clone https://github.com/facebookresearch/pifuhd

## Δοκιμή της εφαρμογής με την δική μας φωτογραφία

In [ ]:
cd /content/pifuhd/sample_images

**Μπορείς να ανεβάσεις την δική σου εικόνα πατώντας το παρακάτω κουμπί**. Αν θες απλά να δοκιμάσεις την εφαρμογή με μία έτοιμη demo εικόνα, μπορείς να παραλείψεις το επόμενο κουμπί.

**ΣΗΜΕΙΩΣΗ:** Για καλύτερα αποτελέσματα, θα πρέπει να βάλεις μία εικόνα που να έχει:
- Υψηλή ανάλυση (μεγαλύτερη από 500x500 pixels)
- Να φαίνεται ένα μόνο άτομο στην φωτογραφία 
- Το άτομο της φωτογραφίας να φαίνεται ολόκληρο από το πρόσωπο έως τα πόδια.
- Η φωτογραφία να έχει καλό φωτισμό
- Το άτομο της φωτογραφίας να είναι στραμμένο ευθεία στην κάμερα

In [ ]:
from google.colab import files

filename = list(files.upload().keys())[0]

In [ ]:
import os

try:
  image_path = '/content/pifuhd/sample_images/%s' % filename
except:
  image_path = '/content/pifuhd/sample_images/test.png' # example image
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output pathes
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

In [ ]:
cd /content

## Τα παρακάτω κουμπιά θα κόψουνε την εικόνα αυτόματα στα κατάλληλα σημεία

In [ ]:
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

In [ ]:
cd /content/lightweight-human-pose-estimation.pytorch/

In [ ]:
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs, demo=True)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)

## Κατεβάζουμε το εκπαιδευμένο μοντέλο

In [ ]:
cd /content/pifuhd/

In [ ]:
!sh ./scripts/download_trained_model.sh

## Τρέχουμε την εφαρμογή PIFuHD!


In [ ]:
# Warning: all images with the corresponding rectangle files under -i will be processed. 
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

# seems that 256 is the maximum resolution that can fit into Google Colab. 
# If you want to reconstruct a higher-resolution mesh, please try with your own machine. 

## Εγκαθιστούμε το απαραίτητο πακέτο για την δημιουργία του 3D μοντέλου

In [ ]:
# This command takes a few minutes
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

## Παράγουμε το 3D μοντέλο και κάνουμε προεπισκόπηση του αποτελέσματος σε ένα βίντεο

In [ ]:
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

**ΣΗΜΕΙΩΣΗ:** Αν θες να εξάγεις το πραγματικό αρχείο του 3D μοντέλου, μπορείς να το βρεις στην λίστα των αρχείων στα δεξια (<-), στον φάκελο:
**/content/pifuhd/results/pifuhd_final/recon/**